# Optimization of k-$\epsilon$ in the Pitz and Daily Case

## Problem description
The pitzDaily case in OpenFOAM is based on the experimental work of Pitz and Daily [1983] which explores the turbulent flow of gases in a combustion chamber. The problem geometry has a backward facing step near the inlet and a tapered nozzle near the outlet with boundary conditions $U = [10, 0 , 0]$ m s$^{-1}$, $\nabla P=0$Pa at the inlet, and $\nabla U=0$m s$^{-1}$, $P=0$Pa at the outlet.

For the Reynolds-Averaged Navier Stokes case, using the simpleFOAM steady-state solver, the turbulence is represented by a $k-\epsilon$ turbulence model which has five coefficients. In addition, the boundary condition for $\epsilon$, i.e the turbulent kinetic energy decay, must be specified. For the Pitz and Daily case, a length scale argument is used to derive a value of 14.855 m$^2$ s$^{-2}$. Following the work of Shirzadi et al. [2017], one of the free parameters $\sigma_\epsilon$ can be calculated from the the other free parameters as

$\sigma_\epsilon = \frac{k^2}{C_{\mu}^{1/2}(C_2 - C_1)}$.

Additionally, the authors also found that for their problem $\sigma_k$ does not significantly affect their simulation and so set that parameter to 1.

Hence the RANS model has four free parameters, one related to the decay of turbulent kinetic energy $\epsilon$ and three related to the $k-\epsilon$ model, $C_\mu$, $C_1$, and $C_2$.

## Goal of the exercise
A common goal of RANS modelling is to replicate the time-averaged behavior of a turbulent case (e.g. from a LES simulation). Often this is done by trying to select values for the free parameters of the RANS case such that metrics of accuracy from the LES and RANS cases are the same. For the Pitz and Daily case, a common metric used to gauge accuracy is the pressure gradient between the outlet and the inlet: $\nabla P = P_{outlet} - P_{inlet}$. Given that the boundary condition for the inlet pressure is zero, the pressure gradient reduces to $\nabla P = P_{outlet}$. Assuming that only the free parameters of the turbulence model-related problems affect the pressure in the RANS case, the accuracy of the RANS model can be quantified as the squared difference between the outlet $P$ from the LES and RANS models. Note here, that to match the scales correctly, we use the large-scale $P$ from the LES model averaged over the last 0.1 seconds of the that simulation as the truth $P_{mean} \approx 1.9 \textrm{Pa}$. Hence, we can pose the tuning of the RANS model as attempting to minimize the following equation:

$f(k, C_\mu, C_1, C_2) = \left( P_{RANS}(k, C_\mu, C_1, C_2) - 1.9 \right)^2$,
where $P_{RANS}$ is the average outlet pressure calculated from the simpleFOAM pitzDaily case.

## Methodology
The penalty function described previously is a computationally expensive calculation because $P_{RANS}$ must be evaluated for every combination of the free parameters. Additionally, traditional gradient-based optimization schemes requires an estimate of the derivatives of the penalty function. The calculation of these derivatives requires an adjoint model and can of similar cost to the fordward solver.

Bayesian optimization is a popular methodology for so-called, blackbox optimization problems, e.g. problems where the only information emitted by function is the output value given a set of inputs. It is also particularly well-suited for applications where the function to be evaluated is expensive to calculate. Instead of directly optimizing the penalty function itself, Bayesian optimization builds a model of the penalty function based on Gaussian processes. The optimization step balances the desire to reduce the uncertainty of the Gaussian processes (referred to as "exploration") and also optimizing the model of the penalty function (referred to as "exploitation").

This exploitation and exploration paradigm lends itself well to parallel execution of the penalty function. A number of exploration/exploitation functions, which suggest the next set of parameters to evaluate, can be called at each stage of optimization. Additionally, many of these functions can suggest multiple sets of parameters. In combination this practically means that at each iteration of the Bayesian Optimization loop, multiple search directions can be explored.

## Implementation

Here we use the Bayesian optimization routine from Scikit-Learn combined with SmartSim to find the optimal set of parameters for the RANS pitzDaily case such that the modelled outlet pressures match those from the LES case. To do this, we take advantage of SmartSim's ability to configure and launch ensembles of simulations and execute them in parallel. While this is a relatively simple problem, it is analogous to many more complex cases. This notebook provides a starting-point for those seeking to perform similar experiments.

In [40]:
from smartredis import Client
import skopt
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['SMARTSIM_LOG_LEVEL'] = 'quiet'
import fluidfoam

### Defining the parameters, default values, and the bounds
To begin, we first define the four free parameters that will be optimized giving along with the default values specified in the default pitzDaily case from the OpenFOAM tutorials. The bounds to explore are taken from the paper of Shirzadi et al. [2017]. Additionally, we also define a function to derive the a fifth variable parameter $\sigma_\epsilon$ which depends on three other free parameters.

In [19]:
param_names = [
    'epsilon',
    'Cmu',
    'C1',
    'C2',
]
default_values = [
    14.855, 
    0.09, 
    1.44, 
    1.92, 
]
bounds = [
    [default_values[0]*0.2, default_values[0]*5],
    [0.05, 0.15],
    [1., 1.5],
    [1.5, 3.],
]
def calculate_sigma_epsilon(Cmu, C2, C1, kappa=0.375):
    return np.sqrt(kappa)/(np.sqrt(Cmu)*(C2-C1))

Next we create the SmartSim `Experiment` object which will be used to define the openFOAM experiment and launch the ensemble at every optimization step

In [2]:
from smartsim import Experiment
exp = Experiment("Tune_K-Epsilon")

Next we define the function that will actually calculate the value of the penalty function described previously. We specify `runSettings` which describe how to run the model itself including the location of the executable `simpleFoam` and potentially how many computational resources should be used to run the simulation. Using these `runSettings` we use the input values to the function to create $N$ ensemble members. The `experiment.generate` call ingests the pitzDaily case, creates run directories for each ensemble member and modifies the parameters used in each experiment. `experiment.start` will actually launch each ensemble member (potentially on HPC resources) and then wait until the entire ensemble returns. Lastly, we extract the averaged inlet pressure and calculate the squared difference between the each ensemble member and the 'true' inlet pressure from the LES simulation.

In [31]:
def evaluate_function(values, true_p = 1.9):

    nparams = len(values[0])

    # Turn the values into a list of dictionaries
    params = {}
    for i in range(nparams):
        params[param_names[i]] = [ value[i] for value in values ]
    # Include the derived sigma_epsilon parameter
    params["sigmaEps"] = [
        calculate_sigma_epsilon(
            params['Cmu'][i], 
            params['C2'][i], 
            params['C1'][i]) 
            for i in range(len(values))
        ]

    rs = exp.create_run_settings(
        "/home/users/shao/dev/OpenFOAM/openfoam/platforms/linux64CrayDPInt32Opt/bin/simpleFoam",
    )
    ens = exp.create_ensemble(
        "evaluation",
        params=params,
        perm_strategy='step',
        run_settings=rs
    )
    ens.attach_generator_files(to_configure="pitzDaily")
    exp.generate(ens, overwrite=True, tag="!")
    exp.start(ens, block=True)

    outputs = []
    for model in ens.models:
        try:
            fname = f'{model.path}/postProcessing/avgInlets/0/surfaceFieldValue.dat'
            data = np.loadtxt(fname, skiprows=5)

            error = (true_p+data[-1,1])**2
            if error>1e3:
                error = np.nan
            
            outputs.append( error )
        except:
            outputs.append(np.nan)

    if len(outputs)==1:
        return outputs[0]
    else:
        return outputs

### Running the Bayesian Optimization Loop

Following the documentation from Scikit-Learn, we use the ask-tell paradigm for Bayesian optimization to investigate multiple search directions (in this case 5) for every iteration loop. We filter the outputs from the optimizer to ensure that only simulations which converged will actually be used to inform the Bayesian optimizer. This process is repeated for 10 loops, outputting the set of parameters that currently yields the smallest penalty function.

In [33]:
optimizer = skopt.Optimizer(
    dimensions=bounds,
    random_state=1,
    base_estimator='gp'
)

for i in range(10):
    x = optimizer.ask(n_points = 5)
    y = evaluate_function(x)

    x = np.array(x)
    y = np.array(y)
    valid = np.isfinite(y) & (y < 1e20)
    x = np.array(x)[valid,:]
    y = np.array(y)[valid]
    
    optimizer.tell(x.tolist(),y.tolist())
    min_idx = np.argmin(optimizer.yi)
    print('Parameter     \tDefault\tOptimal\tMin\tMax')
    for i in range(len(param_names)):
        print(f'{param_names[i]}:     \t{default_values[i]}\t{optimizer.Xi[min_idx][i]:.4f}\t{bounds[i][0]:.4f}\t{bounds[i][1]:.4f}')

{'epsilon': [53.07239854125257, 38.13638397906823, 37.12251980914267, 4.703626825563708, 49.80411078277264], 'Cmu': [0.05616517318142233, 0.13889254049717006, 0.11760874977543352, 0.14369741245296255, 0.11191443526329298], 'C1': [1.1884459380869283, 1.0292435929542256, 1.1578212134843728, 1.4146936728062114, 1.458252414272581], 'C2': [1.7184650930962762, 1.8835422267410677, 2.9474360413114518, 1.7016921234869211, 2.2508573923853263], 'sigmaEps': [4.875178451826281, 1.9233860470091335, 0.9977833389271873, 5.628745433664125, 2.309488395659705]}
Parameter     	Default	Optimal	Min	Max
epsilon:     	14.855	38.1364	2.9710	74.2750
Cmu:     	0.09	0.1389	0.0500	0.1500
C1:     	1.44	1.0292	1.0000	1.5000
C2:     	1.92	1.8835	1.5000	3.0000
{'epsilon': [51.711379692796484, 31.73882451305044, 64.11046407195722, 11.784715845846332, 5.481323960813238], 'Cmu': [0.08346065452010827, 0.12464378847238289, 0.144723312701606, 0.09351649030619424, 0.07069013628998014], 'C1': [1.0147056533501622, 1.0219645416

### Optimization results
After the optimization loop has completed, we see that the penalty function has reduced from 12.1977 Pa$^2$ to 31 $\mu$Pa, close to the accuracy the convergence criterion of the solver itself. The most dramatic change is the value of $\epsilon$ which increases to a value of 31.7 from 14.855. This is not unexpected as the scaling argument used to derive the value originally is intended to provide an order of magnitude estimate. The other parameters are well within the values reported in the values of Sherzadi et al. [2017]. 

In [38]:
print('Parameter     \tDefault\tOptimal\tMin\tMax')
for i in range(len(param_names)):
    print(f'{param_names[i]}:     \t{default_values[i]}\t{optimizer.Xi[min_idx][i]:.4f}\t{bounds[i][0]:.4f}\t{bounds[i][1]:.4f}')
print('-'*12)
print(f'Error:     \t{default_accuracy:.4f}\t{optimizer.yi[min_idx]:.4e}')

Parameter     	Default	Optimal	Min	Max
epsilon:     	14.855	31.7127	2.9710	74.2750
Cmu:     	0.09	0.1055	0.0500	0.1500
C1:     	1.44	1.0639	1.0000	1.5000
C2:     	1.92	2.3314	1.5000	3.0000
------------
Error:     	12.1977	3.1282e-05


## Summary

This notebook has demonstrated how SmartSim can be combined with OpenFOAM to implement a Bayesian optimization workflow that seeks to find a set of turbulence parameters that allows a RANS variant of the pitzDaily case to match that of its LES counterpart. It uses SmartSim's configuration and ensemble launching capabilities to explore multiple different sets of coefficients at every optimization iteration loop. For this case, the optimization loop executes completes within 5 minutes, requiring a total of 50 evaluation of the pitzDaily case to converge to the optimal solution. Both the OpenFOAM case and the optimizer used could be changed out to build a similar workflow, but tailored to the tools and applications of the users case.